<a href="https://colab.research.google.com/github/ZhongyuGuo/M4R/blob/main/Other_methods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import re
import numpy as np
from sklearn.linear_model import LogisticRegression
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import GridSearchCV
import sklearn

In [2]:
from google.colab import drive
drive.mount("/content/drive")
df=pd.read_csv("/content/drive/My Drive/M4R/papers2019_200000.csv")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
TRAIN_FRACTION=0.7

In [4]:
# List of categories
catlist=['math','cs','quant-ph','hep-ex','hep-th','astro-ph','nucl-ex','nucl-th']

In [5]:
def format_text(text):
    """Add spaces around punctuation and remove references to images/citations."""

    # Add spaces around punctuation
    text = re.sub(r'(?<=[^\s0-9])(?=[.,;?])', r' ', text)

    # Remove references to figures
    text = re.sub(r'\((\d+)\)', r'', text)

    text=re.sub(' the ',' ',text)
    text=re.sub(' of ',' ',text)
    text=re.sub(' and ',' ',text)
    text=re.sub(' a ',' ',text)
    text=re.sub(' in ',' ',text)
    text=re.sub(' to ',' ',text)
    text=re.sub(' with ',' ',text)
    text=re.sub(' for ',' ',text)
    text=re.sub(' by ',' ',text)
    text=re.sub(' on ',' ',text)
    text=re.sub(' as ',' ',text)
    text=re.sub(' an ',' ',text)
    text=re.sub(' at ',' ',text)
    text=re.sub(' we ',' ',text)
    text=re.sub(' is ',' ',text)
    text=re.sub(' this ',' ',text)
    text=re.sub(' are ',' ',text)
    text=re.sub(' which ',' ',text)
    text=re.sub(' be ',' ',text)
    text=re.sub(' it ',' ',text)
    text=re.sub(' that ',' ',text)
    text=re.sub(' from ',' ',text)
    text=re.sub(' can ',' ',text)
    text=re.sub(' these ',' ',text)
    text=re.sub(' our ',' ',text)
    text=re.sub(' has ',' ',text)
    text=re.sub(' have ',' ',text)
    text=re.sub('.We ','.',text)
    text=re.sub('.That ','.',text)
    text=re.sub('.The ','.',text)
    text=re.sub('.From ','.',text)
    text=re.sub('.Our ','.',text)
    text=re.sub('.In ','.',text)
    text=re.sub('.These ','.',text)
    text=re.sub('.This ','.',text)
    text=re.sub(',that ',',',text)

    # Remove double spaces
    text = re.sub(r'\s\s', ' ', text)
    
    return text

In [6]:
def make_sequences(texts,
                   training_length=50,
                   lower=True,
                   filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'):
    """Turn a set of texts into sequences of integers"""

    # Create the tokenizer object and train on texts
    tokenizer = Tokenizer(lower=lower, filters=filters)
    tokenizer.fit_on_texts(texts)

    # Create look-up dictionaries and reverse look-ups
    word_idx = tokenizer.word_index
    idx_word = tokenizer.index_word
    num_words = len(word_idx) + 1
    word_counts = tokenizer.word_counts

    print(f'There are {num_words} unique words.')

    # Convert text to sequences of integers
    features = tokenizer.texts_to_sequences(texts)

    categories=list(set(text['categories'].values))
    labels=np.zeros(len(text))
    for i in range(len(text)):
      ind=categories.index(text.iloc[i,1])
      labels[i]=ind
    return word_idx, idx_word, num_words, word_counts, features,labels


In [7]:
def create_train_valid(features,
                       labels,
                       train_fraction=TRAIN_FRACTION):
    """Create training and validation features and labels."""

    # Randomly shuffle features and labels
    features, labels = shuffle(features, labels, random_state=RANDOM_STATE)

    # Decide on number of samples for training
    train_end = int(train_fraction * len(labels))

    train_features = np.array(features[:train_end])
    valid_features = np.array(features[train_end:])

    train_labels = labels[:train_end]
    valid_labels = labels[train_end:]

    # Convert to arrays
    X_train, X_valid = np.array(train_features), np.array(valid_features)
    y_train, y_valid = np.array(train_labels), np.array(valid_labels)

    # Memory management
    import gc
    gc.enable()
    del features, train_features, labels, valid_features, train_labels, valid_labels
    gc.collect()

    return X_train, X_valid, y_train, y_valid

In [8]:
RANDOM_STATE = 50

In [10]:
# Categories to classify
train_acc_list_lr=[]
test_acc_list_lr=[]
train_acc_list_rf=[]
test_acc_list_rf=[]
train_acc_list_svm=[]
test_acc_list_svm=[]
f1_lr=[]
f1_rf=[]
f1_svm=[]
cat1=catlist[1]
df1=df[df['categories']==cat1][0:1500].reset_index(drop=True)
for j in range(2,8):
  print(j)
  cat2=catlist[j]
  # Obtain corresponding abstarcts
  df2=df[df['categories']==cat2][0:1500].reset_index(drop=True)
  text=df1.append(df2).reset_index(drop=True)
  formatted = []

  # Iterate through all the original abstracts
  for a in text['abstracts']:
      formatted.append(format_text(a))
  TRAINING_LENGTH = 50
  filters = '.,!"#$%&()*+/:<=>@[\\]^_`{|}~\t\n'
  word_idx, idx_word, num_words, word_counts,features,labels = make_sequences(
      formatted, TRAINING_LENGTH, lower=True, filters=filters)
  #Make all sequences to same length
  pad = len(max(features, key=len))
  features=[i + [0]*(pad-len(i)) for i in features]
  features1=np.array(features)
  # Define Training and Test sets
  X_input=np.zeros([2000,len(features[0])])
  X_input[0:1000,:]=features1[0:1000,:]
  X_input[1000:2000]=features1[1500:2500,:]
  X_test=np.zeros([1000,len(features[0])])
  X_test[0:500,:]=features1[1000:1500,:]
  X_test[500:1000]=features1[2500:3000,:]
  y_input=np.zeros(2000)
  y_input[0:1000]=labels[0:1000]
  y_input[1000:2000]=labels[1500:2500]
  y_test=np.zeros([1000])
  y_test[0:500]=labels[1000:1500]
  y_test[500:1000]=labels[2500:3000]
  from sklearn.utils import shuffle

  X_train, X_valid, y_train, y_valid = create_train_valid(X_input, y_input)
  #Preprecoess the training data and test features
  from sklearn import preprocessing
  scaler = preprocessing.StandardScaler().fit(X_train)
  X_train_scaled=scaler.transform(X_train)
  scaler1 = preprocessing.StandardScaler().fit(X_test)
  X_test_scaled=scaler1.transform(X_test)

  #LR
  # Parameters to search for the optimal ones
  parameters_LR = {'C':np.linspace(0.001,0.01,19),'solver':['lbfgs','sag','liblinear','newton-cg']}
  #GridSeach for Logistic Regression
  lr=LogisticRegression(max_iter=200000)
  lr_optimal=GridSearchCV(lr,parameters_LR,cv=5,n_jobs=4)
  lr_optimal.fit(X_train_scaled,y_train)
  print('Best params:',lr_optimal.best_params_)
  #Save scores on train and test set
  training_predict =lr_optimal.predict(X_train_scaled)
  test_predict=lr_optimal.predict(X_test_scaled)
  from sklearn.metrics import accuracy_score,f1_score
  train_acc=accuracy_score(y_train,training_predict)
  test_acc=accuracy_score(y_test,test_predict)
  train_acc_list_lr.append(train_acc)
  test_acc_list_lr.append(test_acc)
  train_f1=f1_score(y_train,training_predict)
  test_f1=f1_score(y_test,test_predict)
  f1_lr.append(test_f1)

  #RF
  from sklearn.ensemble import RandomForestClassifier
  # Parameter to search for thr optimal ones
  parameters_RF = {'n_estimators':np.linspace(10,120,12,dtype='int'),'max_depth':np.linspace(10,50,5,dtype='int'),'max_features':np.linspace(5,10,5,dtype='int')}
  rfc=RandomForestClassifier()
  rfc_optimal=GridSearchCV(rfc,parameters_RF,cv=5,n_jobs=8)
  rfc_optimal.fit(X_train_scaled,y_train)
  print('Hyperparameter with biggest impact:',rfc_optimal.best_params_)
  #Show train and test accuracy
  training_predict =rfc_optimal.predict(X_train_scaled)
  test_predict=rfc_optimal.predict(X_test_scaled)
  train_acc=accuracy_score(y_train,training_predict)
  test_acc=accuracy_score(y_test,test_predict)
  train_acc_list_rf.append(train_acc)
  test_acc_list_rf.append(test_acc)
  train_f1=f1_score(y_train,training_predict)
  test_f1=f1_score(y_test,test_predict)
  f1_rf.append(test_f1)

  #SVM
  from sklearn.svm import SVC
  parameters_SVM = {'kernel':('linear','poly','rbf'),'degree':np.linspace(2,8,7),'gamma':('scale','auto'),'C':np.linspace(0.001,0.01,10)}
  svm=SVC()
  svm_optimal=GridSearchCV(svm,parameters_SVM,cv=5,n_jobs=8)
  svm_optimal.fit(X_train_scaled,y_train)

  print('Hyperparameter with biggest impact:',svm_optimal.best_params_)

  training_predict =svm_optimal.predict(X_train_scaled)
  test_predict=svm_optimal.predict(X_test_scaled)
  train_acc=accuracy_score(y_train,training_predict)
  test_acc=accuracy_score(y_test,test_predict)
  train_acc_list_svm.append(train_acc)
  test_acc_list_svm.append(test_acc)
  train_f1=f1_score(y_train,training_predict)
  test_f1=f1_score(y_test,test_predict)
  f1_svm.append(test_f1)

2
There are 22693 unique words.
Best params: {'C': 0.0015, 'solver': 'sag'}
Hyperparameter with biggest impact: {'max_depth': 40, 'max_features': 10, 'n_estimators': 110}
Hyperparameter with biggest impact: {'C': 0.002, 'degree': 2.0, 'gamma': 'scale', 'kernel': 'linear'}
3
There are 20146 unique words.
Best params: {'C': 0.004, 'solver': 'lbfgs'}
Hyperparameter with biggest impact: {'max_depth': 30, 'max_features': 10, 'n_estimators': 50}
Hyperparameter with biggest impact: {'C': 0.001, 'degree': 2.0, 'gamma': 'scale', 'kernel': 'linear'}
4
There are 21583 unique words.
Best params: {'C': 0.003, 'solver': 'lbfgs'}
Hyperparameter with biggest impact: {'max_depth': 20, 'max_features': 10, 'n_estimators': 120}
Hyperparameter with biggest impact: {'C': 0.002, 'degree': 2.0, 'gamma': 'scale', 'kernel': 'linear'}
5
There are 24426 unique words.
Best params: {'C': 0.0025, 'solver': 'lbfgs'}
Hyperparameter with biggest impact: {'max_depth': 30, 'max_features': 8, 'n_estimators': 120}
Hyperpar

In [11]:
train_acc_list_lr

[0.6642857142857143,
 0.875,
 0.7435714285714285,
 0.7042857142857143,
 0.795,
 0.7035714285714286]

In [12]:
test_acc_list_lr

[0.644, 0.837, 0.747, 0.589, 0.792, 0.639]

In [13]:
f1_lr

[0.6008968609865472,
 0.8409756097560975,
 0.761994355597366,
 0.5641569459172853,
 0.7999999999999999,
 0.6130760986066452]

In [14]:
train_acc_list_rf

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0]

In [15]:
test_acc_list_rf

[0.692, 0.878, 0.763, 0.65, 0.836, 0.724]

In [16]:
f1_rf

[0.7274336283185842,
 0.8723849372384939,
 0.7398463227222833,
 0.6641074856046065,
 0.8197802197802199,
 0.7299412915851272]

In [17]:
train_acc_list_svm

[0.6728571428571428,
 0.8721428571428571,
 0.755,
 0.7128571428571429,
 0.8071428571428572,
 0.6842857142857143]

In [18]:
test_acc_list_svm

[0.636, 0.843, 0.746, 0.577, 0.811, 0.648]

In [19]:
f1_svm

[0.5687203791469195,
 0.8480154888673765,
 0.7665441176470587,
 0.5387131952017448,
 0.8123138033763654,
 0.5981735159817352]